# Multiple Linear Regression/Logistic Regression Models in PyTorch and Tensorflow

In [ ]:
import numpy as np
from numpy.random import uniform, binomial

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

# Generate example data
np.random.seed(42)  # Set seed for reproducibility

%precision %.4f

In [ ]:
from sklearn.datasets import make_classification

features, targets = make_classification(    
                                        n_samples = 300,
                                        n_features = 2,
                                        n_informative = 1,
                                        n_redundant = 0,
                                        n_clusters_per_class = 1,
                                        random_state = 42
                                        )


#plt.style.use('fivethirtyeight')
plt.scatter(features[:,0], features[:,1], c=targets)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()


In [ ]:
## logistic function
def logistic(x):
    return 1 / (1 + np.exp(-x))


xx = np.linspace(-10, 10)
plt.plot(xx, logistic(xx))

In [ ]:
n_sample = 100
a = 1.5
b = -4

x = uniform(1, 5, size=n_sample)
x = np.sort(x)

q = logistic(a * x + b) # Linear predictor is a * x + b, 
                        # Link function is logit function 
y = binomial(n=1, p=q) # Probability distribution is binomial distribution (Bernoulli distribution can be other option)
plt.scatter(x, y,  s=10, alpha=0.9)

In [ ]:
x

In [ ]:
y

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

### STEP 1: LOADING DATASET

In [2]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████| 9912422/9912422 [00:06<00:00, 1572352.34it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████| 28881/28881 [00:00<00:00, 327404.39it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████| 1648877/1648877 [00:02<00:00, 702029.16it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 1539809.96it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



### STEP 2: MAKING DATASET ITERABLE

In [3]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

### STEP 3: CREATE MODEL CLASS

In [4]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

### STEP 4: INSTANTIATE MODEL CLASS

In [5]:
input_dim = 28*28
output_dim = 10

model = LogisticRegressionModel(input_dim, output_dim)

### STEP 5: INSTANTIATE LOSS CLASS

In [6]:
criterion = nn.CrossEntropyLoss()

### STEP 6: INSTANTIATE OPTIMIZER CLASS

In [7]:
learning_rate = 0.001

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### STEP 7: TRAIN THE MODEL

In [8]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        images = images.view(-1, 28*28).requires_grad_()
        labels = labels

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # 100 x 10
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = images.view(-1, 28*28).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                # 100 x 1
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            # Print Loss
            print('Iteration: {}, Loss: {}, Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 1.8704924583435059. Accuracy: 66.25
Iteration: 1000. Loss: 1.5236175060272217. Accuracy: 75.85
Iteration: 1500. Loss: 1.3793977499008179. Accuracy: 78.95
Iteration: 2000. Loss: 1.2366455793380737. Accuracy: 80.94
Iteration: 2500. Loss: 1.0039645433425903. Accuracy: 81.93
Iteration: 3000. Loss: 1.0315463542938232. Accuracy: 82.81
